<a href="https://colab.research.google.com/github/adithya1102/Adithya1102.github.io/blob/main/restaurant_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =========================================================
# 🍽️ IMPROVED REAL-TIME RECOMMENDER — CATEGORY-AWARE
# =========================================================

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# -------------------------------
# 1️⃣ LOAD DATASET
# -------------------------------
df = pd.read_csv("dish_dataset.csv")

# Rename if needed
possible_cols = [c for c in df.columns if 'dish' in c.lower()]
if possible_cols and "dish_name" not in df.columns:
    df.rename(columns={possible_cols[0]: "dish_name"}, inplace=True)

# -------------------------------
# 2️⃣ ADD CATEGORY + EXTRA FEATURES
# -------------------------------
def get_category(name):
    name = name.lower()
    if "soup" in name:
        return "soup"
    elif any(k in name for k in ["samosa","tikka","pakora","roll","kebab","fries","bhaji","lollipop"]):
        return "starter"
    elif any(k in name for k in ["masala","curry","makhani","paneer","chicken","dal","biryani"]):
        return "main"
    elif any(k in name for k in ["kheer","jamun","kulfi","halwa","jalebi","falooda","rasmalai","payasam"]):
        return "dessert"
    elif any(k in name for k in ["juice","mocktail","coffee","tea"]):
        return "juice"
    return "other"

df["category"] = df["dish_name"].apply(get_category)

# Add extra differentiating features
def spice_level(name):
    if any(k in name.lower() for k in ["vindaloo","kolhapuri","chettinad","spicy","hot"]):
        return 3
    elif any(k in name.lower() for k in ["masala","tikka","curry"]):
        return 2
    else:
        return 1

def protein_type(name):
    if "paneer" in name.lower(): return "paneer"
    if "chicken" in name.lower(): return "chicken"
    if "mutton" in name.lower(): return "mutton"
    if "fish" in name.lower(): return "fish"
    if "shrimp" in name.lower() or "prawn" in name.lower(): return "seafood"
    return "veg"

df["spice_level"] = df["dish_name"].apply(spice_level)
df["protein_type"] = df["dish_name"].apply(protein_type)

# One-hot encode new features
df = pd.get_dummies(df, columns=["protein_type"], dtype=int)

# -------------------------------
# 3️⃣ CATEGORY ORDER
# -------------------------------
category_sequence = ["soup","main", "starter", "main", "dessert", "juice"]

# -------------------------------
# 4️⃣ RECOMMENDER FUNCTION
# -------------------------------
def recommend_next(user_history):
    # Determine next category
    ordered_cats = [get_category(d) for d in user_history]
    next_cat = None
    for c in category_sequence:
        if c not in ordered_cats:
            next_cat = c
            break
    if not next_cat:
        print("🎉 Full course completed!")
        return None

    # Find ordered dish rows
    ordered_idx = [df[df["dish_name"].str.lower() == d.lower()].index[0]
                   for d in user_history if d.lower() in df["dish_name"].str.lower().values]

    if not ordered_idx:
        print("⚠️ Dish not found in dataset.")
        return None

    # Compute average taste vector
    features = [col for col in df.columns if col not in ["dish_name", "category"]]
    user_pattern = df.loc[ordered_idx, features].mean().values.reshape(1, -1)

    # Compute similarity only among next category dishes
    subset = df[df["category"] == next_cat]
    subset_features = subset[features].values
    sims = cosine_similarity(user_pattern, subset_features).flatten()

    subset["similarity"] = sims
    recs = subset.sort_values(by="similarity", ascending=False).head(5)
    return recs[["dish_name", "category", "similarity"]]

# -------------------------------
# 5️⃣ REAL-TIME INTERACTION
# -------------------------------
print("\n🍽️ Real-Time Dish Recommender\n")
print("Type the dish you ordered. Type 'exit' to quit.\n")

history = []

while True:
    user_input = input("Enter ordered dish: ").strip()
    if user_input.lower() == "exit":
        print("\n🍴 Session Ended.")
        break

    if user_input not in df["dish_name"].values:
        print("⚠️ Dish not found. Try again.")
        continue

    history.append(user_input)
    recs = recommend_next(history)
    if recs is not None:
        print("\n🍛 Recommended Next Dishes:")
        print(recs.to_string(index=False))



🍽️ Real-Time Dish Recommender

Type the dish you ordered. Type 'exit' to quit.

Enter ordered dish: Paneer Tikka


/tmp/ipython-input-2968604571.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset["similarity"] = sims



🍛 Recommended Next Dishes:
           dish_name category  similarity
     Hot & Sour Soup     soup    0.758175
   Mulligatawny Soup     soup    0.741249
Vegetable Clear Soup     soup    0.741249
         Tomato Soup     soup    0.667124
           Corn Soup     soup    0.667124
Enter ordered dish: Paneer Tikka


/tmp/ipython-input-2968604571.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset["similarity"] = sims



🍛 Recommended Next Dishes:
           dish_name category  similarity
     Hot & Sour Soup     soup    0.758175
   Mulligatawny Soup     soup    0.741249
Vegetable Clear Soup     soup    0.741249
         Tomato Soup     soup    0.667124
           Corn Soup     soup    0.667124
Enter ordered dish: Mushroom Soup


...


🍛 Recommended Next Dishes:
     dish_name category  similarity
   Kofta Curry     main    0.943456
Paneer Dopiaza     main    0.934477
  Matar Paneer     main    0.934477
  Palak Paneer     main    0.934477
 Murgh Makhani     main    0.932278
Enter ordered dish: Lachha Paratha


/tmp/ipython-input-2968604571.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset["similarity"] = sims



🍛 Recommended Next Dishes:
        dish_name category  similarity
      Kofta Curry     main    0.951534
    Murgh Makhani     main    0.946836
Vegetable Biryani     main    0.946836
     Chana Masala     main    0.933593
    Bhindi Masala     main    0.933593
Enter ordered dish: Jalebi


/tmp/ipython-input-2968604571.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset["similarity"] = sims



🍛 Recommended Next Dishes:
        dish_name category  similarity
      Kofta Curry     main    0.954130
    Murgh Makhani     main    0.953983
Vegetable Biryani     main    0.953983
     Chana Masala     main    0.933568
    Bhindi Masala     main    0.933568


In [ ]:
# =========================================================
# 🍽️ SMART REAL-TIME DISH RECOMMENDATION SYSTEM (2 MAIN COURSES)
# =========================================================

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load dataset
df = pd.read_csv("dish_dataset.csv")

# -------------------------------
# Define dish categories
dish_category = {
    "soup": list(df[df["liquid_diet"] == 1]["dish_name"]),
    "starter": list(df["dish_name"].iloc[0:20]),
    "main_course_1": list(df["dish_name"].iloc[20:53]),
    "main_course_2": list(df["dish_name"].iloc[20:53]),  # same dishes, but second round recommendation
    "bread_rice": list(df["dish_name"].iloc[53:73]),
    "dessert": list(df["dish_name"].iloc[92:100]),
}

# Ordered meal flow
sequence_order = ["soup", "starter", "main_course_1", "main_course_2", "bread_rice", "dessert"]

# -------------------------------
# Helper function: find dish category
def find_category(dish_name):
    for cat, dishes in dish_category.items():
        if dish_name in dishes:
            return cat
    return None

# -------------------------------
# Personalized weights tracker
user_preferences = {
    "vegetarian": 0,
    "non_vegetarian": 0,
    "low_calories": 0,
    "spicy": 0,
    "sweet": 0
}

def update_preferences(dish_name):
    global user_preferences
    row = df[df["dish_name"] == dish_name].iloc[0]
    if row["vegetarian"] == 1:
        user_preferences["vegetarian"] += 1
    else:
        user_preferences["non_vegetarian"] += 1
    if row["low_calories"] == 1:
        user_preferences["low_calories"] += 1
    if any(x in dish_name.lower() for x in ["vindaloo", "kolhapuri", "chettinad", "tikka", "65"]):
        user_preferences["spicy"] += 1
    if any(x in dish_name.lower() for x in ["kheer", "jamun", "rasmalai", "jalebi", "kulfi", "halwa"]):
        user_preferences["sweet"] += 1

# -------------------------------
# Recommendation logic
def recommend_next(user_history):
    if not user_history:
        return []

    last_dish = user_history[-1]
    current_stage = find_category(last_dish)

    if current_stage is None:
        print(f"⚠️ Unknown dish: {last_dish}")
        return []

    # Find the next stage
    current_index = sequence_order.index(current_stage)
    if current_index < len(sequence_order) - 1:
        next_stage = sequence_order[current_index + 1]
    else:
        print("🍽️ You’ve reached the end of meal recommendations!")
        return []

    # Compute similarity
    feature_cols = [c for c in df.columns if c != "dish_name"]
    dish_vector = df[df["dish_name"] == last_dish][feature_cols].values
    all_vectors = df[feature_cols].values
    sim_scores = cosine_similarity(dish_vector, all_vectors)[0]
    df["similarity"] = sim_scores

    # Filter by next stage and user preference
    next_dishes = df[df["dish_name"].isin(dish_category[next_stage])]
    next_dishes = next_dishes.sort_values("similarity", ascending=False)

    # Apply lightweight preference adjustment
    if user_preferences["vegetarian"] > user_preferences["non_vegetarian"]:
        next_dishes = next_dishes[next_dishes["vegetarian"] == 1]
    elif user_preferences["non_vegetarian"] > user_preferences["vegetarian"]:
        next_dishes = next_dishes[next_dishes["vegetarian"] == 0]

    # Exclude already ordered dishes
    next_dishes = next_dishes[~next_dishes["dish_name"].isin(user_history)]

    return next_dishes[["dish_name", "similarity"]].head(5)

# -------------------------------
# Real-time simulation
print("🍽️ Smart Real-Time Dish Recommendation System (2 Main Courses)\n")
user_history = []

while True:
    dish = input("Enter ordered dish (or 'exit' to stop): ").strip()
    if dish.lower() == "exit":
        break
    if dish not in df["dish_name"].values:
        print("❌ Dish not found in menu. Try again.")
        continue

    user_history.append(dish)
    update_preferences(dish)
    print(f"\n🧾 Order History: {user_history}")
    print(f"🎯 User Preferences: {user_preferences}")

    recommendations = recommend_next(user_history)
    if recommendations.empty:
        print("No more recommendations available.\n")
    else:
        print("\n🍛 Recommended Next Dishes:")
        print(recommendations.to_string(index=False))
    print("\n" + "=" * 70 + "\n")


🍽️ Smart Real-Time Dish Recommendation System (2 Main Courses)

Enter ordered dish (or 'exit' to stop): Shrimp Garlic

🧾 Order History: ['Shrimp Garlic']
🎯 User Preferences: {'vegetarian': 0, 'non_vegetarian': 1, 'low_calories': 0, 'spicy': 0, 'sweet': 0}

🍛 Recommended Next Dishes:
           dish_name  similarity
          Crab Curry    1.000000
     Shrimp Vindaloo    0.948683
Shrimp Butter Garlic    0.948683
   Prawn Malai Curry    0.948683
          Fish Curry    0.942809


Enter ordered dish (or 'exit' to stop): kheer
❌ Dish not found in menu. Try again.
Enter ordered dish (or 'exit' to stop): Paneer Tikka

🧾 Order History: ['Shrimp Garlic', 'Paneer Tikka']
🎯 User Preferences: {'vegetarian': 1, 'non_vegetarian': 1, 'low_calories': 0, 'spicy': 1, 'sweet': 0}

🍛 Recommended Next Dishes:
           dish_name  similarity
        Matar Paneer    1.000000
        Palak Paneer    1.000000
      Paneer Dopiaza    1.000000
Paneer Butter Masala    0.951326
 Paneer Tikka Masala    0.951326


In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Load dataset
df = pd.read_csv("dish_dataset.csv")

# Normalize column name just in case
df.columns = [col.strip() for col in df.columns]

# --- CATEGORY ORDER FLOW ---
CATEGORY_FLOW = {
    "soup": "starter",
    "starter": "main_course",
    "main_course": "main_course_2",
    "main_course_2": "dessert_juice",
    "dessert_juice": "farewell",
    "farewell": None  # End of flow
}

# --- MAPPING OF CATEGORIES TO DATASET COLUMNS ---
CATEGORY_COLUMN_MAP = {
    "soup": ["soup"],
    "starter": ["starter"],
    "main_course": ["main_course"],
    "main_course_2": ["main_course"],
    "dessert_juice": ["dessert", "juice"],
    "farewell": ["tea", "coffee"]
}

# Create a feature matrix (numeric only)
X = df.select_dtypes(include=['int64', 'float64'])
similarity_matrix = cosine_similarity(X)

# Utility: get category of dish
def get_category(dish_name):
    if dish_name not in df['dish_name'].values:
        return None
    row = df[df['dish_name'] == dish_name].iloc[0]
    for category, cols in CATEGORY_COLUMN_MAP.items():
        for c in cols:
            if row.get(c, 0) == 1:
                return category
    return None

# Utility: recommend dishes based on current pattern
def recommend_dish(current_dishes):
    available_dishes = df['dish_name'].values

    # Compute combined pattern vector (mean of selected dishes)
    selected_indices = [df.index[df['dish_name'] == d][0] for d in current_dishes if d in df['dish_name'].values]
    if not selected_indices:
        print("⚠️ No valid dishes found in current history.")
        return []

    user_vector = X.iloc[selected_indices].mean().values.reshape(1, -1)

    # Get similarity
    sim_scores = cosine_similarity(user_vector, X).flatten()

    # Sort dishes by similarity
    ranked_indices = sim_scores.argsort()[::-1]

    # Determine next category
    last_dish = current_dishes[-1]
    last_cat = get_category(last_dish)
    next_cat = CATEGORY_FLOW.get(last_cat)

    if not next_cat:
        print("🍵 Final suggestion phase completed. Thank you for dining with us!")
        return []

    valid_cols = CATEGORY_COLUMN_MAP[next_cat]
    mask = df[valid_cols].any(axis=1)

    # Exclude already ordered dishes
    filtered = df[mask & ~df['dish_name'].isin(current_dishes)]

    # Sort filtered dishes by similarity
    filtered = filtered.iloc[ranked_indices]

    recommendations = filtered['dish_name'].head(3).tolist()
    return recommendations

# --- REAL-TIME LOOP ---
print("\n🍽️ Smart Real-Time Restaurant Recommendation System 🍛\n")
print("👉 Type the dish name to simulate an order. Type 'exit' to stop.\n")

ordered_dishes = []

while True:
    user_input = input("Enter ordered dish: ").strip()
    if user_input.lower() == 'exit':
        print("\n✅ Session ended. Hope your dining experience was pleasant!")
        break

    if user_input not in df['dish_name'].values:
        print("❌ Dish not found in the menu. Please try again.\n")
        continue

    ordered_dishes.append(user_input)

    # Get next dish recommendations
    recommendations = recommend_dish(ordered_dishes)
    if not recommendations:
        break

    print(f"\n🍽️ Based on your current selection ({', '.join(ordered_dishes)}), we recommend next:\n➡️ {', '.join(recommendations)}\n")


In [ ]:
#RECOMMENDATION BASED ON EXISTING DATA.


# =========================================================
# 🍽️ SMART REAL-TIME DISH RECOMMENDATION SYSTEM (2 MAIN COURSES)
# =========================================================

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load dataset
df = pd.read_csv("dish_dataset.csv")

# -------------------------------
# Define dish categories
dish_category = {
    "soup": list(df[df["liquid_diet"] == 1]["dish_name"]),
    "starter": list(df["dish_name"].iloc[0:20]),
    "main_course_1": list(df["dish_name"].iloc[20:53]),
    "main_course_2": list(df["dish_name"].iloc[20:53]),  # same dishes, but second round recommendation
    "bread_rice": list(df["dish_name"].iloc[53:73]),
    "dessert": list(df["dish_name"].iloc[92:100]),
}

# Ordered meal flow
sequence_order = ["soup", "starter", "main_course_1", "main_course_2", "bread_rice", "dessert"]

# -------------------------------
# Helper function: find dish category
def find_category(dish_name):
    for cat, dishes in dish_category.items():
        if dish_name in dishes:
            return cat
    return None

# -------------------------------
# Personalized weights tracker
user_preferences = {
    "vegetarian": 0,
    "non_vegetarian": 0,
    "low_calories": 0,
    "spicy": 0,
    "sweet": 0
}

def update_preferences(dish_name):
    global user_preferences
    row = df[df["dish_name"] == dish_name].iloc[0]
    if row["vegetarian"] == 1:
        user_preferences["vegetarian"] += 1
    else:
        user_preferences["non_vegetarian"] += 1
    if row["low_calories"] == 1:
        user_preferences["low_calories"] += 1
    if any(x in dish_name.lower() for x in ["vindaloo", "kolhapuri", "chettinad", "tikka", "65"]):
        user_preferences["spicy"] += 1
    if any(x in dish_name.lower() for x in ["kheer", "jamun", "rasmalai", "jalebi", "kulfi", "halwa"]):
        user_preferences["sweet"] += 1

# -------------------------------
# Recommendation logic
def recommend_next(user_history):
    if not user_history:
        return []

    last_dish = user_history[-1]
    current_stage = find_category(last_dish)

    if current_stage is None:
        print(f"⚠️ Unknown dish: {last_dish}")
        return []

    # Find the next stage
    current_index = sequence_order.index(current_stage)
    if current_index < len(sequence_order) - 1:
        next_stage = sequence_order[current_index + 1]
    else:
        print("🍽️ You’ve reached the end of meal recommendations!")
        return []

    # Compute similarity
    feature_cols = [c for c in df.columns if c != "dish_name"]
    dish_vector = df[df["dish_name"] == last_dish][feature_cols].values
    all_vectors = df[feature_cols].values
    sim_scores = cosine_similarity(dish_vector, all_vectors)[0]
    df["similarity"] = sim_scores

    # Filter by next stage and user preference
    next_dishes = df[df["dish_name"].isin(dish_category[next_stage])]
    next_dishes = next_dishes.sort_values("similarity", ascending=False)

    # Apply lightweight preference adjustment
    if user_preferences["vegetarian"] > user_preferences["non_vegetarian"]:
        next_dishes = next_dishes[next_dishes["vegetarian"] == 1]
    elif user_preferences["non_vegetarian"] > user_preferences["vegetarian"]:
        next_dishes = next_dishes[next_dishes["vegetarian"] == 0]

    # Exclude already ordered dishes
    next_dishes = next_dishes[~next_dishes["dish_name"].isin(user_history)]

    return next_dishes[["dish_name", "similarity"]].head(5)

# -------------------------------
# Real-time simulation
print("🍽️ Smart Real-Time Dish Recommendation System (2 Main Courses)\n")
user_history = []

while True:
    dish = input("Enter ordered dish (or 'exit' to stop): ").strip()
    if dish.lower() == "exit":
        break
    if dish not in df["dish_name"].values:
        print("❌ Dish not found in menu. Try again.")
        continue

    user_history.append(dish)
    update_preferences(dish)
    print(f"\n🧾 Order History: {user_history}")
    print(f"🎯 User Preferences: {user_preferences}")

    recommendations = recommend_next(user_history)
    if recommendations.empty:
        print("No more recommendations available.\n")
    else:
        print("\n🍛 Recommended Next Dishes:")
        print(recommendations.to_string(index=False))
    print("\n" + "=" * 70 + "\n")